In [16]:
import re
import time
import json
import yaml
import random
import itertools
import dicttoxml, xmltodict

from pprint import pprint as pp
from functools import wraps, partial

In [17]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start) * 1000
        print(self.message.format(elapsed_time))
        
class A():
    def __enter__(self):
        print('__enter__')
    
    def __exit__(self, type, value, traceback):
        print('__exit__')

# with A():
#     raise ValueError('WRONG!')


## Magic attributes/methods

In [18]:
class A: # class A(object):
    """
    This is A
    """
    def __init__(self, name):
        self.name = name
    
    def foo(self):
        """
        foo
        """
        print('foo', self.name, self)

    
a = A('test')
b = A('test2')
# a.foo()
A.foo(a)
A.foo(b)

print(a)
print(b)
# print(dir(a))
# print(dir(A))
# print(dir(object))

foo test <__main__.A object at 0x7f6558728a00>
foo test2 <__main__.A object at 0x7f65587281f0>


In [19]:
print(a.__class__)
print(A.__class__, A.__bases__)
print(A.__doc__)
print(A.foo.__doc__)
help(A.foo)

<class '__main__.A'>
<class 'type'> (<class 'object'>,)

    This is A
    

        foo
        
Help on function foo in module __main__:

foo(self)
    foo



In [20]:
print(a.__str__())
print(str(a))

dir(a)
a.__dir__()

# xxx(a)
# a.__xxx__()

v = 42
print(v + 1)
print(v.__add__(1))

class Employee:
    def __init__(self, first_name=None, last_name=None, email=None):
        self._first_name = first_name
        self._last_name = last_name
        self._email = email
    
    def __bool__(self):
        return bool(self._first_name or \
               self._last_name or \
               self._email)
            
e = Employee()


if e:
    print('Not empty')
    # do some work
    
if e is not None: # None ~= null
    print('Is not None')
    # do some work

43
43
Is not None


#### operator overloading

In [21]:
class Q:
    def __init__(self, **params):
        self._params = params
    
    def __or__(self, other):
        self._params.update(other._params)
        return self

#     def __and__(self, other):
#         self._params.update(other._params)
#         return self
    
    def __str__(self):
        result = ''
        for k, v in  self._params.items():
            if result:
                result += ' OR '
#             if result:
#                 result += ' OR '
            result += f'{k}={repr(v)}'
        return result

filter = Q()
filter |= Q(first_name='John')
filter |= Q(last_name='Gonzalez')
filter |= Q(stuff=True)
filter |= Q(age=42)

print(filter)

first_name='John' OR last_name='Gonzalez' OR stuff=True OR age=42


#### `__dict__`

In [22]:
class A:
    attr1 = 'attr1' # static 
    
    def __init__(self, obj_attr1, obj_attr2):
        self.obj_attr1 = obj_attr1
        self.obj_attr2 = obj_attr2
        
a = A(1, 'abc')
a2 = A(2, 'xyz')

print(a.obj_attr1, id(a.obj_attr1))
print(a2.obj_attr1, id(a2.obj_attr1))
print(a.attr1, id(a.attr1))
print(a2.attr1, id(a2.attr1))

1 9752160
2 9752192
attr1 140073253134128
attr1 140073253134128


In [23]:
pp('Obj attrs: ')
pp(a.__dict__)
print()
pp('Class attrs: ')
pp(A.__dict__)

'Obj attrs: '
{'obj_attr1': 1, 'obj_attr2': 'abc'}

'Class attrs: '
mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__init__': <function A.__init__ at 0x7f65590b1430>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'attr1': 'attr1'})


In [24]:
print(a.obj_attr1)
print(a.__dict__['obj_attr1'])
a.__dict__['obj_attr1'] = 42
print(a.obj_attr1)
A.attr1 = 'XYZ'
print(A.__dict__['attr1'])
print(a.attr1)

1
1
42
XYZ
XYZ


In [25]:
b = A(1, 'abc')
print(id(a.attr1), id(b.attr1))
print(id(a.obj_attr1), id(b.obj_attr1))
print(id(a.obj_attr2), id(b.obj_attr2))

140073252743920 140073252743920
9753472 9752160
140073382397040 140073382397040


In [26]:
a = A(1024, 'abc')
b = A(1024, 'abc'.upper().lower())
# c = A(1024, 'ab'+'c')
print(id(a.attr1), id(b.attr1))
print(id(a.obj_attr1), id(b.obj_attr1))
print(id(a.obj_attr2), id(b.obj_attr2))

140073252743920 140073252743920
140073252771440 140073252771856
140073382397040 140073252361328


In [27]:
b.attr1 = 42
print(id(a.attr1), id(b.attr1))

140073252743920 9753472


In [28]:
pp('Obj attrs: ')
pp(b.__dict__)
print()
pp('Class attrs: ')
pp(A.__dict__)

'Obj attrs: '
{'attr1': 42, 'obj_attr1': 1024, 'obj_attr2': 'abc'}

'Class attrs: '
mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__init__': <function A.__init__ at 0x7f65590b1430>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'attr1': 'XYZ'})


In [29]:
# Monkey patchihttp://localhost:8889/notebooks/lesson_classes.ipynb#ng
a.xxxx = 42
pp('Obj attrs: ')
pp(a.__dict__)
print(a.xxxx)


def super_lib():
    def super_func():
        print('foo')
    super_func = super_func
    
def profile(f):
    from time import time
    def deco(*args):
        start = time()
        f(*args)
        print('Elapsed: ', time()-start)
    return deco

print(id(super_lib.super_func))
super_lib.super_func = profile(super_lib.super_func)
print(id(super_lib.super_func))

'Obj attrs: '
{'obj_attr1': 1024, 'obj_attr2': 'abc', 'xxxx': 42}
42


AttributeError: 'function' object has no attribute 'super_func'

In [ ]:
a.foo = lambda x: print(x)

In [ ]:
a.foo('bar')
pp('Obj attrs: ')
pp(a.__dict__)

#### get/set attributes

In [ ]:
class A:
    def __getattr__(self, name):
        print('__getattr__')
#         if name in self.__dict__:
#             value = self.__dict__[name]
#         elif name in self.__class__.__dict__:
#             value = self.__class__.__dict__[name]
#         else:
#             value = super().__getattr__(name)                    
#         return value
        return 42
    
    def __setattr__(self, name, value):
        print('__setattr__')
        self.__dict__[name] = value + 1
        
    def __delattr__(self, name):
        if name in self.__dict__:
            del self.__dict__[name]
            
a = A()
a.boo = 42
print(a.boo)

In [ ]:
def foo():
    print('foo')
foo()
foo.__call__()

class A:
    def __call__(self):
        print('__call__')
    
a = A()
a()


In [ ]:
class lazy_object:
    '''
    Class for deferred instantiation of objects.  Init is called
    only when the first attribute is either get or set.
    '''

    def __init__(self, callable, *args, **kw):
        '''
        callable -- Class of objeсt to be instantiated or functionnn to be called
        *args -- arguments to be used when instantiating object
        **kw  -- keywords to be used when instantiating object
        '''
        self.__dict__['callable'] = callable
        self.__dict__['args'] = args
        self.__dict__['kw'] = kw
        self.__dict__['obj'] = None

    def init_obj(self):
        '''
        Instantiate object if not already done
        '''
        if self.obj is None:
            self.__dict__['obj'] = self.callable(*self.args, **self.kw)

    def __getattr__(self, name):
        self.init_obj()
        return getattr(self.obj, name)

    def __setattr__(self, name, value):
        self.init_obj()
        setattr(self.obj, name, value)

    def __len__(self):
        self.init_obj()
        return len(self.obj)

    def __getitem__(self, idx):
        self.init_obj()
        return self.obj[idx]


In [ ]:
class A:
    def __init__(self, num_elem):
        self.attr1 = list(range(num_elem))
        
a = lazy_object(A, num_elem=10**8)

print(a)

In [ ]:
with timer('Elapsed: {}ms'):
   type(a.attr1)

with timer('Elapsed: {}ms'):
   type(a.attr1)

#### `__call__`

In [ ]:
def foo():
    print('foo')

foo.__call__()

In [ ]:
class A:
    def __call__(self):
        print('called on object')

a = A()
a()

### Properties, hiding data

In [ ]:
class A: # before
    
    def __init__(self, attr1=None, attr2=None):
        self.attr1 = attr1 # protected (client)
        self.attr2 = attr2 # protected (client)

class A:
    
    def __init__(self, attr1=None, attr2=None):
        self._attr1 = attr1 # protected (client)
        self._attr2 = attr2 # protected (client)
        self.__attr3 = attr2 # private (client&child)

    @property
    def attr1(self):
        print('get attr1')
        return self._attr1
        
    @attr1.setter
    def attr1(self, value):
        print('set attr1')
        if value > 0:
            self._attr1 = value
        else:
            raise ValueError('Invalid data')
       
    @property
    def attr2(self):
        print('get attr2')
        return self._attr2

a = A(attr2='abc')
a.attr1 = 42
print(a.attr1)
#a.attr1 = -1
print(a.attr2)
#a.attr2 = 'ABC'
#print(a.__attr3)

print(a.__dict__)
#print(a._A__attr3)
print(a._A__attr3)
a.__attr3 = 'ABC'
print(a.__attr3)
print(a.__dict__)


In [ ]:
a._attr2 = 'ABC'
print(a._attr2)



## Inheritance

In [ ]:
class A:
    def __init__(self):
        self.attr1 = 'attrA'
        self.attrX = 'attrX'

    def foo(self):
        print('Hello from A.foo()')


class A1(A):
    def __init__(self):
        super().__init__()

class A2(A):
    
    def __init__(self):
        print('A2() called')
        
class B1(A1):
    pass

a1 = A1()
print(a1.attr1)

a2 = A2()
a2.foo()
print(a2.attr1)

In [ ]:
class A:

    def __init__(self):
        super().__init__()
    
    def foo(self):
        
        print('Hello from A.foo()')


class A1(A):
    def __init__(self):
        super().__init__()
        self.attrA1 = 'attrA1'
        
    def foo_a1(self):
        print('foo_a1() called')

        
class A2(A):
    def __init__(self):
        super().__init__()
        self.attrA2 = 'attrA2'

    def foo_a2(self):
        print('foo_a2() called')

        
a1 = A1()
a2 = A2()
print(a1.attrA1)
print(a2.attrA2)
a1.foo()
a2.foo()

a1.foo_a1()
a2.foo_a2()

In [ ]:
  
class B(A1, A2):
    def __init__(self):
        super().__init__()
        self.attr1 = 'attrB'
        self.attr2 = 'attr2B'

b = B()
print(b.__dict__)


In [ ]:
print(B.mro())

In [ ]:
# Links:
# https://en.wikipedia.org/wiki/C3_linearization
# http://python-history.blogspot.com/2010/06/method-resolution-order.html

### Multiple Inheritance -> Mixins
#### A mix-in is a small class: 
- only defines a set of additional methods/attributes
- don't require their __init__ constructor to be called.


In [ ]:
class JsonMixin(object):
    
    @classmethod
    def from_json(cls, data):
        obj = cls()
        attrs = json.loads(data)
        vars(obj).update(attrs)
        return obj
    
    def to_json(self):
        return json.dumps(vars(self))
    

class XMLMixin(object):
    
    @classmethod
    def from_xml(cls, data):
        obj = cls()
        attrs = xmltodict.parse(data)
        vars(obj).update(attrs)
        return obj
    
    def to_xml(self):
        return dicttoxml.dicttoxml(vars(self)).decode()
    

class YamlMixin(object):
    
    @classmethod
    def from_yaml(cls, data):
        kwargs = yaml.load(data)
        return cls(**kwargs)
    
    def to_yaml(self):
        return yaml.dump(vars(self))

In [ ]:
class C(JsonMixin, XMLMixin, YamlMixin, B):
    pass
c = C()

print(c.to_json())
print(c.to_xml())
print(c.to_yaml())

print()
print(C.from_json(c.to_json()).to_json())
print(C.from_json(c.to_json()).to_json() == c.to_json())
# print(C.from_xml(c.to_xml()))

#### Static methods

In [ ]:
class A1:
    
    attr = 'attr_A'
    
    def bar(self):
        print('foo')
    
    @staticmethod
    def foo():
        print('foo: ', A1.attr)
        

    @classmethod
    def foo_class(cls):
        print('foo_class: ', cls.attr)

class B(A):
    attr = 'attr_B'
   
# A.bar()
A.foo()
A.foo_class()

B.foo()
B.foo_class()

### slots

In [ ]:
class A:
    __slots__ = ['attr1', 'attr2']

a = A()
a.attr1 = 42
a.attr2 = '42'
print(a.attr1, a.attr2)

In [ ]:
#a.attr3 = 'abc'

In [ ]:
class A1:
    def __init__(self, attr1, attr2):
        self.attr1 = attr1
        self.attr2 = attr2

a1 = A1(42, '42')
print(a1.attr1, a1.attr2)

In [ ]:
from sys import getsizeof
print(getsizeof(a), a.__slots__, getsizeof(a.__slots__))
print(getsizeof(a1), a1.__dict__, getsizeof(a1.__dict__))

In [ ]:
with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A()
        a.attr1 = 42
        a.attr2 = '42'
        _ = a.attr1, a.attr2

with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A1(42, '42')
        _ = a.attr1, a.attr2


In [ ]:
# NamedTuple()

### Descriptors

In [ ]:
# Technically, descriptor is a class that supports the following methods: __set__[, __get__],__delete__

In [ ]:
class A:
    attr1 = (int, 0)
    attr2 = (str, '')

    def __getattribute__(self, name): # always called
        if name == '__dict__':
            return super().__getattribute__(name)
        obj_attrs = self.__dict__
        cls_attrs = vars(type(self))
        if name not in obj_attrs:
            if name in cls_attrs:
                _, default = cls_attrs[name]
                self.__dict__[name] = default
        return self.__dict__[name]

#     def __getattr__(self, name): # lookup fallback method
#         print('__getattr__', name)
#         obj_attrs = vars(self) # self.__dict__
#         cls_attrs = vars(type(self)) # self.__class__.__dict__
#         if name not in obj_attrs:
#             if name in cls_attrs:
#                 _, default = cls_attrs[name]
#                 self.__dict__[name] = default
#         return self.__dict__[name]
    
    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

    def __delattr__(self, name):
        del self.__dict__[name]

In [ ]:
a = A()
a1 = A()
print(a.attr1)
print(a.attr2)

a.attr1 = 32
a.attr2 = '42'

a1.attr1 = 33
a1.attr2 = 'xyz'

print(a.attr1, a1.attr1)
print(a.attr2, a1.attr2)


Technically, descriptor is a class that supports the following methods: __set__[, __get__],__delete__

In [ ]:
class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        #print('Retrieving', self.name, id(obj))
        return obj.__dict__[self.name] #self.val

    def __set__(self, obj, val):
        #print('Updating', self.name, id(obj))
        self.val = val
        obj.__dict__[self.name] = val
        
    def __delete__(self, obj):
        #print('Deleting', self.name, id(obj))
        self.val = None
        
class A:
    attr = Attribute(name='attr')
    #attr = 'DEMO'
 
a = A()
print(id(a))
a.attr = 42
print(a.attr)
#del a.attr

b = A()
print(id(b))
b.attr = 43
print(a.attr)
print(b.attr)
#del b.attr

# a1 = A()
# a.attr = 44
# a1.attr = 45
# print(a.attr)
# print(a1.attr)
# del a1.attr

In [ ]:
# a.attr1 = '42'
a.attr2 = 42

In [35]:
class TypeCheckerMixin:

    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

class A(TypeCheckerMixin):
    attr1 = (int, 0)
    attr2 = (str, '')
    attr3 = (bool, False)

a = A()

a.attr1 = 42
#a.attr1 = '42'

# But what if we want to check range for ints, regex amtch for strings, isclose() for floats?

In [34]:

class Descriptor:
    def __init__(self, name=None, default=None):
        self.name = name
        self.default = default

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

    def __get__(self, instance, objtype):
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = self.default
        return instance.__dict__[self.name]

    def __delete__(self, instance):
        raise AttributeError("Can't delete")


class Typed(Descriptor):
    type_ = object
    extra_methods = []
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('Expected %s' % self.type_)
        super().__set__(instance, value)


# Specialized types
class Numeric(Typed):
    extra_methods = ['gt', 'gte']

    def gt(instance_value, value):
        return instance_value > value

    def gte(instance_value, value):
        return instance_value >= value

class Integer(Numeric):
    type_ = int

class Float(Numeric):
    type_ = float
    extra_methods = Numeric.extra_methods + ['isclose']

    def isclose(instance_value, value):
        import math
        return math.isclose(instance_value, value)

class String(Typed):
    type_ = str
    extra_methods = ['startswith', 'endswith', 'contains']

    def startswith(instance_value, value):
        return instance_value.startswith(value)

    def endswith(instance_value, value):
        return instance_value.endswith(value)

    def contains(instance_value, value):
        return value in instance_value

In [36]:
class A:
    attr1 = Integer(name='attr1')

a = A()
a.attr1 = 32
# a.attr1 = '32'

In [37]:
# Value checking
class Positive(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)


# More specialized types
class PosInteger(Integer, Positive):
    pass


class PosFloat(Float, Positive):
    pass


# Length checking
class Sized(Descriptor):
    def __init__(self, *args, maxlen, **kwargs):
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)

    def __set__(self, instance, value):
        if len(value) > self.maxlen:
            raise ValueError('Too big')
        super().__set__(instance, value)


class SizedString(String, Sized):
    pass


# Pattern matching
class Regex(Descriptor):
    def __init__(self, *args, pattern, **kwargs):
        self.pattern = re.compile(pattern)
        super().__init__(*args, **kwargs)

    def __set__(self, instance, value):
        if not self.pattern.match(value):
            raise ValueError('Invalid string')
        super().__set__(instance, value)


class SizedRegexString(SizedString, Regex):
    pass


In [38]:
class A:
    attr1 = PosInteger(default=42)
    attr2 = PosFloat()
    attr3 = SizedRegexString(maxlen=11, pattern='\d{3}-\d{7}')

a = A()
print(a.attr1)
a.attr1 = 32
print(a.attr1)
a.attr2 = 0.1
a.attr3 = '067-9372129'

a1 = A()
a1.attr1 = 5
print(a1.attr1)

print(id(a.attr1))
print(id(a1.attr1))

print(PosInteger.mro())

42
32
5
140073262912624
9752288
[<class '__main__.PosInteger'>, <class '__main__.Integer'>, <class '__main__.Numeric'>, <class '__main__.Typed'>, <class '__main__.Positive'>, <class '__main__.Descriptor'>, <class 'object'>]


### Metaclasses


Let's debug our code

In [39]:
def debug(func):
    '''
    A simple debugging decorator
    '''
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'{msg} run took {time.time()-start} ms')
        return result
    return wrapper

In [40]:
@debug
def foo():
    time.sleep(1)
    
foo()

foo run took 1.0007898807525635 ms


In [41]:
class A():
    
    @debug
    def foo(self):
        time.sleep(random.random())

a = A()
a.foo()

A.foo run took 0.3248271942138672 ms


In [42]:
class A():
    
    @debug
    def foo(self):
        time.sleep(random.random())
        
    @debug
    def bar(self):
        time.sleep(random.random())
        
    @debug
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()

A.foo run took 0.1924147605895996 ms
A.bar run took 0.6133582592010498 ms
A.baz run took 0.8557188510894775 ms


In [43]:
def debugmethods(cls):
    '''
    Apply a decorator to all callable methods of a class
    '''
    for name, val in vars(cls).items(): # cls.__dict__
        if callable(val):
            setattr(cls, name, debug(val))

    setattr(cls, 'xxx', 42)

    return cls

# A.foo = debug(A.foo)

# # A.bar = debug(A.bar)
# # setattr(A, 'bar', debug(A.bar))

# A.baz = debug(A.baz)
# help(callable)

In [44]:
@debugmethods
class A():
    
    def foo(self):
        time.sleep(random.random())
        
    def bar(self):
        time.sleep(random.random())
        
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()
print(a.xxx)

A.foo run took 0.8580012321472168 ms
A.bar run took 0.4554758071899414 ms
A.baz run took 0.2630341053009033 ms
42


In [45]:
class B(A):
    def foo_b(self):
        time.sleep(random.random())
        
    def bar_b(self):
        time.sleep(random.random())
        
    def baz_b(self):
        time.sleep(random.random())
        
b = B()
b.foo_b()
b.bar_b()
b.baz_b()


Let's use metaclasses to entire eirarchy with debug decorators

In [46]:
A = type('A', (object,), {'attr1': 42, 'attr2': 'abc'})

print(type(A), id(A))
print(A.__dict__)
a = A()
print(type(a))
print(a.attr1)
print(a.attr2)
a.attr1 = 43
print(a.attr1)

<class 'type'> 21345136
{'attr1': 42, 'attr2': 'abc', '__module__': '__main__', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
<class '__main__.A'>
42
abc
43


In [47]:
class A:
    attr1 = 42
    attr2 = 'abc'
    
print(type(A), id(A))
print(A.__dict__)
a = A()
print(type(a))
print(a.attr1)
print(a.attr2)
a.attr1 = 43
print(a.attr1)

<class 'type'> 21261760
{'__module__': '__main__', 'attr1': 42, 'attr2': 'abc', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
<class '__main__.A'>
42
abc
43


In [48]:
A = type('A',
         (object,),
         {'attr1': 42,
          'attr2': 'abc',
          'foo': lambda self: self.attr1+1}
        )
a = A()
a.foo()

43

<img src="https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png">

In [ ]:
print(type(a))
print(a.__class__)

In [ ]:
print(type(A))
print(A.__class__)

# not parent, but creator. Compare with
print(A.__bases__)

In [ ]:
print(type(type))
print(type.__class__)

# not parent, but creator. Compare with
print(type.__bases__)

In [ ]:
print(type(type(type(type(type(type))))))
print(type.__class__)

In [ ]:
class A(metaclass=type):
    def foo(self):
        print('foo')
        
a = A()
a.foo()
# print(a.bar)

In [49]:
class mytype(type):
    '''
    Metaclass default implementation
    '''
    def __new__(metacls, clsname, bases=None, clsdict=None):
        cls = super().__new__(metacls, clsname, bases, clsdict)
        cls.bar = 42
        setattr(cls, 'objects', 'XYZ')
        return cls

    # def __init__(cls, name, bases, dct):
    #     super().__init__(name, bases, dct)

In [50]:
class A(metaclass=mytype):
    def foo(self):
        print('foo')

class B(A):
    pass

a = A()
a.foo()
print(a.bar)


b = B()
b.foo()
print(b.bar)
print(b.objects)

foo
42
foo
42
XYZ


In [54]:
class ModelMeta(type):

    def __new__(metacls, clsname, bases=None, clsdict=None):
        cls = super().__new__(metacls, clsname, bases, clsdict)
        extra_attrs = []
        for attr_name, attr_value in cls.__dict__.items():
            if isinstance(attr_value, Typed):
                extra_attrs += [
                    (attr_name, extra_method, getattr(attr_value.__class__, extra_method))
                    for extra_method in attr_value.extra_methods
                ]

        for attr, extra, func in extra_attrs:
            setattr(
                cls,
                f'{attr}__{extra}',
                lambda self, value, attr=attr, func=func: func(getattr(self, attr), value)
            )

        return cls

class Employee(metaclass=ModelMeta):
    first_name = SizedString(name='first_name', default='John', maxlen=32)
    last_name = SizedString(name='last_name', maxlen=64)
    age = PosInteger(name='age', default=42)
    salary = PosFloat(name='salary')
    phone_number = SizedRegexString(name='phone_number', maxlen=11, pattern='\d{3}-\d{7}')

In [2]:

class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name, id(obj))
        return obj.__dict__[self.name]  # self.val

    def __set__(self, obj, val):
        print('Updating', self.name, id(obj))
        self.val = val
        obj.__dict__[self.name] = val

    def __delete__(self, obj):
        # print('Deleting', self.name, id(obj))
        self.val = None


class A:
    attr = Attribute(name='attr')
    # attr = 'DEMO'
    
a = A()
a.attr=10

Updating attr 140452546327024


In [52]:
emp = Employee()
print(emp.first_name)
print(emp.first_name__startswith('J'))
print(emp.age__gte(42))
emp.age = 10
print(emp.age__gt(42))

John
True
True
False


In [ ]:
class debugmeta(type):
    '''
    Metaclass that applies debugging to methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        clsobj = debugmethods(clsobj)
        return clsobj


In [ ]:
class A(metaclass=debugmeta):
    
    def foo(self):
        time.sleep(random.random())
        
    def bar(self):
        time.sleep(random.random())
        
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()

In [ ]:
class B(A):
    def foo_b(self):
        time.sleep(random.random())
        
    def bar_b(self):
        time.sleep(random.random())
        
    def baz_b(self):
        time.sleep(random.random())
        
b = B()
b.foo_b()
b.bar_b()
b.baz_b()